# TU Chemnitz SmartLoc


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Stanford-NavLab/gnss_lib_py/blob/main/notebooks/tutorials/parsers/smartloc.ipynb)

This tutorial shows how to load data from TU Chemnitz's [smartLoc GNSS Dataset](https://www.tu-chemnitz.de/projekt/smartLoc/gnss_dataset.html.en#Datasets).

Load `gnss_lib_py` into the Python workspace

In [ ]:
import gnss_lib_py as glp

In [ ]:
# download cropped SmartLoc data file
glp.make_dir("../data")
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/data/unit_test/smartloc/tu_chemnitz_berlin_1_raw.csv --quiet -nc -O "../data/smartloc.csv"

# load smartLoc data into NavData object
smartloc_data = glp.SmartLocRaw("../data/smartloc.csv")

In [ ]:
# plot the pseudorange over time of each individual satellite
# SBAS 120 is the outlier with its larger orbit
fig = glp.plot_metric(smartloc_data, "gps_millis","raw_pr_m", groupby="sv_id")

In [ ]:
# show the ground truth smartLoc data on a map
fig = glp.plot_map(smartloc_data)
fig.show()

We have also implemented some helper functions to process `SmartlocRaw` instances.
1. NLOS measurements, as specified in the measurements, can be removed easily using the `remove_nlos` method.
2. The ground truth measurements are in WGS-84 frame of reference by default and they can be converted to ECEF using `calculate_gt_ecef`.
3. Ground truth velocity and acceleration values are given as a set of local body-frame scalar value, combined with the heading angle of the vehicle. This can be easily converted to ECEF values using `calculate_gt_vel`.

All of these methods create a copy of the `SmartLocRaw` instance, which is a `NavData` instance with the additional rows calculated.

In [ ]:
# Remove NLOS
import gnss_lib_py.parsers.smartloc as sl
print('Total measurement instances:', len(smartloc_data))
smartloc_data_no_nlos = sl.remove_nlos(smartloc_data)
print('LOS measurement instances', len(smartloc_data_no_nlos))

In [ ]:
# Use wildcard index to show ECEF position row does not exist
try:
    glp.find_wildcard_indexes(smartloc_data, "x_*_m")
except KeyError as no_row_exp:
    print(no_row_exp)

# Compute ECEF ground truth position and show that row exists
smartloc_ecef_gt = sl.calculate_gt_ecef(smartloc_data)
print(glp.find_wildcard_indexes(smartloc_ecef_gt, "x_*_m"))

In [ ]:
# Use wildcard index to show that ECEF velocity rows do not exist
try:
    glp.find_wildcard_indexes(smartloc_data, "vx_*_mps")
except KeyError as no_row_exp:
    print(no_row_exp)

# Use wildcard index to show that body frame velocity exists
print(glp.find_wildcard_indexes(smartloc_data, "v_*_mps"))

# Compute ECEF ground truth velocity and verify row exists
smartloc_vel_gt = sl.calculate_gt_vel(smartloc_data)
print(glp.find_wildcard_indexes(smartloc_vel_gt, "vx_*_mps"))

In [ ]:
print(smartloc_data.rows)